In [4]:
!pip install selenium


In [1]:
#importing libraries
from selenium import webdriver 
from selenium.webdriver import ActionChains 
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException, NoSuchElementException 
import time
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver. support import expected_conditions as EC
import pandas as pd

# 1.Andhra Bus Route Details

In [16]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the APSRTC option
apsrtc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='APSRTC']")
driver.execute_script("arguments[0].click();", apsrtc_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 6):
    if page_number < 6:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(2)
        next_page.click()
        time.sleep(2)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 16']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        AP_SRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
APSRTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(2)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(1)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    # Fully bus items will be loaded and scroll function call
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(2)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in APSRTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore', 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole', 'https://www.redbus.in/bus-tickets/ongole-to-hyderabad', 'https://www.redbus.in/bus-tickets/tirupathi-to-bangalore', 'https://www.redbus.in/bus-tickets/kadapa-to-bangalore', 'https://www.redbus.in/bus-tickets/bangalore-to-tirupathi', 'https://www.redbus.in/bus-tickets/bangalore-to-kadapa', 'https://www.redbus.in/bus-tickets/narsaraopet-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-narsaraopet', 'https://www.redbus.in/bus-tickets/ananthapur-to-bangalore', 'https://www.redbus.in/bus-tickets/bangalore-to-chittoor-andhra-pradesh', 'https://www.redbus.in/bus-tickets/madanapalli-to-bangalore', 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada', 'https:/

In [22]:
df_apsrtc = pd.DataFrame(APSRTC)
df_apsrtc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Chittoor (Andhra Pradesh) to Bangalore Bus,https://www.redbus.in/bus-tickets/chittoor-and...,APSRTC - 9061,Express(Non AC Seater),21:45,03h 25m,01:10,4.0,INR 170,N/A
1,Chittoor (Andhra Pradesh) to Bangalore Bus,https://www.redbus.in/bus-tickets/chittoor-and...,APSRTC - 5524,Express(Non AC Seater),22:45,04h 10m,02:55,4.1,INR 215,N/A
2,Chittoor (Andhra Pradesh) to Bangalore Bus,https://www.redbus.in/bus-tickets/chittoor-and...,APSRTC - 9080,Express(Non AC Seater),23:15,03h 25m,02:40,4.6,INR 170,N/A
3,Chittoor (Andhra Pradesh) to Bangalore Bus,https://www.redbus.in/bus-tickets/chittoor-and...,APSRTC - 5528,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",23:59,05h 16m,05:15,4.3,INR 304,N/A
4,Chittoor (Andhra Pradesh) to Bangalore Bus,https://www.redbus.in/bus-tickets/chittoor-and...,NueGo,Electric A/C Seater (2+2),23:20,05h 25m,04:45,4.0,Starts from\nINR 330 314,N/A
...,...,...,...,...,...,...,...,...,...,...
743,Hyderabad to Chilakaluripet Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,VSN Travels,A/C Seater / Sleeper (2+1),22:10,06h 20m,04:30,2.9,Starts from\nINR 399,N/A
744,Hyderabad to Chilakaluripet Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,SRI MARUTHI TRAVELS,Non A/C Seater / Sleeper (2+1),21:45,07h 45m,05:30,2.6,Starts from\nINR 399,N/A
745,Hyderabad to Chilakaluripet Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,SRI MARUTHI TRAVELS,Non A/C Seater / Sleeper (2+1),21:40,07h 50m,05:30,2.5,Starts from\nINR 399,N/A
746,Hyderabad to Chilakaluripet Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Hriday Tours & Travels,Volvo Multi-Axle A/C Sleeper (2+1),22:00,06h 45m,04:45,1.6,INR 3000,N/A


In [23]:
# Andhra Bus details
# read the csv files
path = r"Z:/Redbus project/APSRTC bus details.csv"
df_apsrtc.to_csv(path,index = False)

# 2.Telugana Bus Route Details

In [9]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(3)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(3)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(3)

#Click on the TSRTC option
tsrtc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='TSRTC']")
driver.execute_script("arguments[0].click();", tsrtc_element)
# driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='TSRTC']").click()
# time.sleep(3)

#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(3)

# Collect route links
links = []
for page_number in range(1, 4):
    if page_number < 4:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(5)
        next_page.click()
        time.sleep(3)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(3)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-16']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        TS_RTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
TS_RTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(5)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(3)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(3)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in TS_RTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/hyderabad-to-srisailam', 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'https://www.redbus.in/bus-tickets/hyderabad-to-khammam', 'https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad', 'https://www.redbus.in/bus-tickets/guntur-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-mancherial', 'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole', 'https://www.redbus.in/bus-tickets/kothagudem-to-hyderabad', 'https://www.redbus.in/bus-tickets/godavarikhani-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-guntur', 'https://www.redbus.in/bus-tickets/hyderabad-to-nirmal', 'https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar', 'https://www.redbus.in/bus-tickets/jagityal-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-kothagudem', 'https://www.redbus.in/bus-tickets/hyderabad-to

In [10]:
df_tsrtc = pd.DataFrame(TS_RTC)
df_tsrtc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Hyderabad to Srisailam Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 4322,Super Luxury (Non AC Seater 2+2 Push Back),19:45,12h 15m,08:00,4.5,INR 442,N/A
1,Hyderabad to Srisailam Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 1254,Super Luxury (Non AC Seater 2+2 Push Back),20:10,12h 20m,08:30,4.1,INR 442,N/A
2,Hyderabad to Srisailam Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 1252,Super Luxury (Non AC Seater 2+2 Push Back),20:15,12h 15m,08:30,4.0,INR 442,N/A
3,Hyderabad to Srisailam Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 1280,Super Luxury (Non AC Seater 2+2 Push Back),20:45,11h 15m,08:00,3.8,INR 442,N/A
4,Hyderabad to Srisailam Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 1045,Rajdhani (AC Semi Sleeper 2+2),21:30,11h 00m,08:30,4.0,INR 524,N/A
...,...,...,...,...,...,...,...,...,...,...
159,Hyderabad to Armoor Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Saini Travels Pvt. Ltd.,A/C Sleeper (2+1),21:30,04h 10m,01:40,3.9,Starts from\nINR 1000,N/A
160,Hyderabad to Armoor Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Vijayant Travels,A/C Sleeper (2+1),21:45,03h 30m,01:15,4.1,INR 2000,N/A
161,Hyderabad to Armoor Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Mahadev Bus Service,Non A/C Seater / Sleeper (2+1),22:00,03h 00m,01:00,4.1,Starts from\nINR 1500,N/A
162,Hyderabad to Armoor Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Mahadev Bus Service,A/C Sleeper (2+1),22:00,03h 30m,01:30,4.1,Starts from\nINR 1000,N/A


In [24]:
# read the csv files
path = r"Z:/Redbus project/tsrtc bus details.csv"
df_tsrtc.to_csv(path,index = False)

# 3.KERALA ROUTES BUS DETAILS

In [27]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the KSRTC option
ksrtc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='KERALA RTC']")
driver.execute_script("arguments[0].click();", ksrtc_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 3):
    if page_number < 3:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-16']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        KSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
KSRTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(3)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in KSRTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kannur', 'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore', 'https://www.redbus.in/bus-tickets/kozhikode-to-kottayam', 'https://www.redbus.in/bus-tickets/kannur-to-bangalore', 'https://www.redbus.in/bus-tickets/thrissur-to-kozhikode', 'https://www.redbus.in/bus-tickets/thiruvananthapuram-to-kozhikode', 'https://www.redbus.in/bus-tickets/kottayam-to-kozhikode']
{'bus_route_name': 'Bangalore to Kozhik

In [28]:
df_ksrtc = pd.DataFrame(KSRTC)
df_ksrtc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1210,SWIFT-HYBRID NON AC SEATER CUM SLEEPER,12:55,08h 10m,21:05,3.8,Starts from\nINR 567,19 Seats available
1,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,07h 39m,21:25,2.7,INR 729,15 Seats available
2,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,10h 00m,01:00,2.9,INR 620,N/A
3,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 380,Swift Deluxe Non AC Air Bus (2+2),16:15,08h 45m,01:00,3.3,INR 567,N/A
4,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,09h 30m,02:15,3.5,INR 640,N/A
...,...,...,...,...,...,...,...,...,...,...
811,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Madhavi Travels,A/C Sleeper (2+1),18:50,08h 10m,03:00,5.0,INR 999 949,N/A
812,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Madhavi Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),23:59,04h 31m,04:30,3.5,INR 999 949,N/A
813,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,07h 19m,02:29,4.1,Starts from\nINR 750,N/A
814,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Ubc travels,A/C Seater/Sleeper (2+1),18:00,09h 00m,03:00,3.7,Starts from\nINR 900,N/A


In [29]:
# Kerala Bus details
# read the csv files
path = r"Z:/Redbus project/KSRTC bus details.csv"
df_ksrtc.to_csv(path,index = False)

# 4.GOA ROUTE BUS DETAILS

In [31]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the TSRTC option
ktcl_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='KTCL']")
driver.execute_script("arguments[0].click();", ktcl_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 5):
    if page_number < 5:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 30']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        KTCL.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
KTCL=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(3)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in KTCL:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-tickets/goa-to-pune', 'https://www.redbus.in/bus-tickets/mumbai-to-goa', 'https://www.redbus.in/bus-tickets/bangalore-to-goa', 'https://www.redbus.in/bus-tickets/belagavi-to-goa', 'https://www.redbus.in/bus-tickets/goa-to-mumbai', 'https://www.redbus.in/bus-tickets/goa-to-bangalore', 'https://www.redbus.in/bus-tickets/goa-to-belagavi', 'https://www.redbus.in/bus-tickets/pandharpur-to-goa', 'https://www.redbus.in/bus-tickets/solapur-to-goa', 'https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra', 'https://www.redbus.in/bus-tickets/goa-to-pandharpur', 'https://www.redbus.in/bus-tickets/goa-to-solapur', 'https://www.redbus.in/bus-tickets/calangute-goa-to-goa-airport', 'https://www.redbus.in/bus-tickets/goa-to-sangola', 'https://www.redbus.in/bus-tickets/goa-airport-to-calangute-goa', 'https://www.redbus.in/bus-tickets/sangola-to-goa', 'https://www.redbus.in/bus-tickets/calangute-goa-to-mopa-airport', 'ht

In [32]:
df_ktcl = pd.DataFrame(KTCL)
df_ktcl

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,10h 00m,05:00,3.6,INR 600,N/A
1,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.8,Starts from\nINR 1100,N/A
2,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.7,Starts from\nINR 1099,N/A
3,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.6,Starts from\nINR 1099,N/A
4,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:15,10h 40m,07:55,4.5,Starts from\nINR 610,N/A
...,...,...,...,...,...,...,...,...,...,...
606,Mopa Airport to Panaji Bus,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),16:00,00h 55m,16:55,3.6,INR 200,N/A
607,Goa to Shirdi Bus,https://www.redbus.in/bus-tickets/goa-to-shirdi,PSR Travels Goa,A/C Sleeper (2+1),19:15,14h 45m,10:00,3.6,Starts from\nINR 1399,N/A
608,Goa to Shirdi Bus,https://www.redbus.in/bus-tickets/goa-to-shirdi,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),19:00,16h 00m,11:00,2.4,Starts from\nINR 1450,N/A
609,Goa to Shirdi Bus,https://www.redbus.in/bus-tickets/goa-to-shirdi,Hans Travels (I) Private Limited,A/C Sleeper (2+1),12:30,15h 45m,04:15,2.2,INR 1400,N/A


In [33]:
# read the csv files
path = r"Z:/Redbus project/KTCL(goa) bus details.csv"
df_ktcl.to_csv(path,index = False)

# 5.Rajasthan Route bus details

In [38]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the RSRTC option
driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='RSRTC']").click()
time.sleep(1)

#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 3):
    if page_number < 3:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        RSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
RSRTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                button.click()
                time.sleep(1)
            except NoSuchElementException:
                continue
    except TimeoutException:
        continue
    # Fully bus items will be loaded and scroll function call
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in RSRTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'https://www.redbus.in/bus-tickets/beawer-to-jaipur', 'https://www.redbus.in/bus-tickets/jaipur-to-jodhpur', 'https://www.redbus.in/bus-tickets/udaipur-to-jodhpur', 'https://www.redbus.in/bus-tickets/sikar-to-jaipur', 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur', 'https://www.redbus.in/bus-tickets/kishangarh-to-jaipur', 'https://www.redbus.in/bus-tickets/kota-rajasthan-to-jaipur', 'https://www.redbus.in/bus-tickets/jaipur-to-pilani', 'https://www.redbus.in/bus-tickets/jodhpur-to-beawer', 'https://www.redbus.in/bus-tickets/jaipur-to-aligarh-uttar-pradesh', 'https://www.redbus.in/bus-tickets/kota-rajasthan-to-udaipur', 'https://www.redbus.in/bus-tickets/sikar-to-bikaner', 'https://www.redbus.in/bus-tickets/jaipur-to-kota-rajasthan', 'https://www.redbus.in/bus-tickets/udaipur-to-pali-rajasthan', 'https://www.redbus.in/bus-tickets/udaipur-to-jaipur', 'https://www.redbus.in/bus-tickets/bikaner-to-sikar', 'https://

In [39]:
df_rsrtc = pd.DataFrame(RSRTC)
df_rsrtc
         

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 160781,Super Luxury Volvo AC Seater Pushback 2+2,00:00,14h 00m,14:00,4.0,INR 460,N/A
1,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 144042,Super Luxury Volvo AC Seater Pushback 2+2,00:00,12h 20m,12:20,3.0,INR 460,N/A
2,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 192566,Semi Deluxe Non AC Seater 2+2,04:45,04h 30m,09:15,4.0,INR 250,N/A
3,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 146314,Super Luxury Volvo AC Seater Pushback 2+2,06:15,04h 00m,10:15,4.0,INR 460,N/A
4,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 185141,Super Luxury Volvo AC Seater Pushback 2+2,07:15,04h 00m,11:15,4.3,INR 460,N/A
...,...,...,...,...,...,...,...,...,...,...
1129,Mathura to Jaipur Bus,https://www.redbus.in/bus-tickets/mathura-to-j...,GV Tours,A/C Seater/Sleeper (2+1),23:30,04h 45m,04:15,3.8,Starts from\nINR 500 475,42 Seats available
1130,Mathura to Jaipur Bus,https://www.redbus.in/bus-tickets/mathura-to-j...,Rinku Travels,A/C Seater / Sleeper (2+1),23:30,05h 52m,05:22,3.5,Starts from\nINR 400,45 Seats available
1131,Mathura to Jaipur Bus,https://www.redbus.in/bus-tickets/mathura-to-j...,Rinku Travels,A/C Seater / Sleeper (2+1),23:35,06h 40m,06:15,3.2,Starts from\nINR 400,33 Seats available
1132,Mathura to Jaipur Bus,https://www.redbus.in/bus-tickets/mathura-to-j...,SHAGUN CARGO & TRAVELS,NON A/C Seater/ Sleeper (2+1),01:15,04h 30m,05:45,2.7,Starts from\nINR 600 540,N/A


In [40]:
# read the csv files
path = r"Z:/Redbus project/rsrtc(Rajasthan) bus details.csv"
df_rsrtc.to_csv(path,index = False)

# 6.South Bengal Routes Bus details

In [5]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the SBSTC option
sbstc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='SBSTC']")
driver.execute_script("arguments[0].click();", sbstc_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 6):
    if page_number < 6:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 16']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        SBSTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
SBSTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(1)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in SBSTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/durgapur-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-burdwan', 'https://www.redbus.in/bus-tickets/kolkata-to-durgapur', 'https://www.redbus.in/bus-tickets/haldia-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-haldia', 'https://www.redbus.in/bus-tickets/midnapore-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal', 'https://www.redbus.in/bus-tickets/kolkata-to-midnapore', 'https://www.redbus.in/bus-tickets/kolkata-to-digha', 'https://www.redbus.in/bus-tickets/kolkata-to-asansol', 'https://www.redbus.in/bus-tickets/kolkata-to-bankura', 'https://www.redbus.in/bus-tickets/jhargram-to-kolkata', 'https://www.redbus.in/bus-tickets/digha-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-nimtouri', 'https://www.redbus.in/bus-tickets/kolkata-to-contai-kanthi', 'https://www.redbus.in/bus-tickets/digha-to-durgapur', 'https://www.redbus.in/bus-tickets/kolkata-to-nandakumar', 'https://www.redbus.in/b

In [6]:
df_sbstc = pd.DataFrame(SBSTC)
df_sbstc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Durgapur (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 13:00 (DGP DEPOT) -...,Non AC Seater (2+3),13:00,03h 30m,16:30,3.5,INR 150,N/A
1,Durgapur (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 12:20 (BEL DEPOT...,Non AC Seater (2+3),13:30,04h 00m,17:30,3.9,INR 155,N/A
2,Durgapur (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - L/S - 12:35 (HOWRAH ...,Non AC Seater (2+3),13:40,03h 30m,17:10,2.6,INR 150,N/A
3,Durgapur (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - 12:00 (BNK DEPOT...,Non AC Seater (2+3),13:50,04h 00m,17:50,3.6,INR 155,N/A
4,Durgapur (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - HABRA - VIA - DUNLOP - 14:05 ...,Non AC Seater (2+3),14:05,03h 30m,17:35,4.4,INR 155,N/A
...,...,...,...,...,...,...,...,...,...,...
513,Kolkata to Ashari Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - ROHINI - VIA - KULTIKRI - 16:2...,Non AC Seater (2+3),16:20,07h 40m,00:00,3.9,INR 82,N/A
514,Kolkata to Ashari Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - A/C - 16:30 (MID DE...,AC Seater (2+2),16:30,07h 30m,00:00,3.9,INR 135,N/A
515,Kolkata to Ashari Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - JHARGRAM - VIA - LODHASULI - 1...,Non AC Seater (2+3),16:50,07h 10m,00:00,3.8,INR 82,N/A
516,Kolkata to Ashari Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) -...,Non AC Seater (2+3),18:00,06h 00m,00:00,4.4,INR 82,N/A


In [7]:
# read the csv files
path = r"Z:/Redbus project/SBSTC(South Bengal) bus details.csv"
df_sbstc.to_csv(path,index = False)

# 7. Himachal Pradesh Route Bus details

In [8]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the HRTC option
hrtc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='HRTC']")
driver.execute_script("arguments[0].click();", hrtc_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 6):
    if page_number < 6:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 16']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        HRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
HRTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(3)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in HRTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/delhi-to-shimla', 'https://www.redbus.in/bus-tickets/shimla-to-delhi', 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh', 'https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh', 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-chandigarh', 'https://www.redbus.in/bus-tickets/chamba-himachal-pradesh-to-chandigarh', 'https://www.redbus.in/bus-tickets/kangra-to-chandigarh', 'https://www.redbus.in/bus-tickets/dharamshala-to-chandigarh', 'https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh', 'https://www.redbus.in/bus-tickets/manali-to-chandigarh', 'https://www.redbus.in/bus-tickets/shimla-to-manali', 'https://www.redbus.in/bus-tickets/palampur-to-chandigarh', 'https://www.redbus.in/bus-tickets/delhi-to-nalagarh', 'https://www.redbus.in/bus-tickets/chandigarh-to-manali', 'https://www.redbus.in/bus-tickets/shimla-to-chandigarh',

In [9]:
df_hrtc = pd.DataFrame(HRTC)
df_hrtc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Delhi to Shimla Bus,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 551,A/C Executive (2+3),15:30,10h 00m,01:30,2.6,INR 632,N/A
1,Delhi to Shimla Bus,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,10h 10m,03:00,4.1,INR 512,N/A
2,Delhi to Shimla Bus,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,10h 16m,04:15,3.0,INR 512,N/A
3,Delhi to Shimla Bus,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 15m,04:50,4.5,INR 912,N/A
4,Delhi to Shimla Bus,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,09h 47m,05:30,3.9,INR 512,N/A
...,...,...,...,...,...,...,...,...,...,...
884,Jahu to Delhi Bus,https://www.redbus.in/bus-tickets/jahoo-to-delhi,HRTC - 700,Himsuta AC Seater Volvo/Scania 2+2,19:00,09h 30m,04:30,4.6,INR 1083,N/A
885,Jahu to Delhi Bus,https://www.redbus.in/bus-tickets/jahoo-to-delhi,HRTC - 749,Ordinary 3+2 Non AC Seater,19:00,10h 30m,05:30,4.3,INR 611,N/A
886,Jahu to Delhi Bus,https://www.redbus.in/bus-tickets/jahoo-to-delhi,HRTC - 750,A/C Executive (2+3),19:30,10h 30m,06:00,4.1,INR 696,N/A
887,Jahu to Delhi Bus,https://www.redbus.in/bus-tickets/jahoo-to-delhi,HRTC - 1107,Ordinary 3+2 Non AC Seater,19:45,08h 30m,04:15,4.0,INR 560,N/A


In [10]:
# read the csv files
path = r"Z:/Redbus project/HRTC(Himachal Pradesh) bus details.csv"
df_hrtc.to_csv(path,index = False)

# 8.Uttar pradesh Route Bus Details

In [10]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the UPSRTC option
upsrtc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='UPSRTC']")
driver.execute_script("arguments[0].click();", upsrtc_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 6):
    if page_number < 6:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 30']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        UPSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
UPSRTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(3)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in UPSRTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/bareilly-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-bareilly', 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi', 'https://www.redbus.in/bus-tickets/lucknow-to-delhi', 'https://www.redbus.in/bus-tickets/lucknow-to-allahabad', 'https://www.redbus.in/bus-tickets/farrukhabad-up-to-delhi', 'https://www.redbus.in/bus-tickets/allahabad-to-lucknow', 'https://www.redbus.in/bus-tickets/sitapur-to-delhi', 'https://www.redbus.in/bus-tickets/gorakhpur-to-lucknow', 'https://www.redbus.in/bus-tickets/agra-to-lucknow', 'https://www.redbus.in/bus-tickets/lucknow-to-agra', 'https://www.redbus.in/bus-tickets/moradabad-to-delhi', 'https://www.redbus.in/bus-tickets/varanasi-to-lucknow', 'https://www.redbus.in/bus-tickets/delhi-to-farrukhabad-up', 'https://www.redbus.in/bus-tickets/aligarh-to-lucknow', 'https://www.redbus.in/bus-tickets/lucknow-to-varanasi', 'https://www.redbus.in/bus-tickets/delhi-to-aligarh-uttar-pradesh', 'https://www.redb

In [11]:
df_upsrtc = pd.DataFrame(UPSRTC)
df_upsrtc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Bareilly (Uttar Pradesh) to Delhi Bus,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0089,Ordinary Non AC Seater 2+3,16:01,07h 58m,23:59,3.3,INR 480,N/A
1,Bareilly (Uttar Pradesh) to Delhi Bus,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0119,Shatabdi AC Seater 2+2,16:16,05h 00m,21:16,3.3,INR 648,N/A
2,Bareilly (Uttar Pradesh) to Delhi Bus,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - KSB0086,Volvo AC Seater 2+2,17:01,05h 30m,22:31,4.1,INR 894,N/A
3,Bareilly (Uttar Pradesh) to Delhi Bus,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0130,Janrath AC Seater 2+3,17:31,05h 00m,22:31,2.8,INR 570,N/A
4,Bareilly (Uttar Pradesh) to Delhi Bus,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0111,Janrath AC Seater 2+2,19:01,04h 44m,23:45,4.4,INR 648,N/A
...,...,...,...,...,...,...,...,...,...,...
1178,Delhi to Haridwar Bus,https://www.redbus.in/bus-tickets/delhi-to-har...,India Tours & Travels (GetBookCab),A/C Seater Push Back (2+2),21:00,06h 00m,03:00,2.4,INR 778 700,N/A
1179,Delhi to Haridwar Bus,https://www.redbus.in/bus-tickets/delhi-to-har...,Dev Darbar Travel Agency,A/C Sleeper (2+1),20:10,10h 35m,06:45,2.4,INR 4999 4499,N/A
1180,Delhi to Haridwar Bus,https://www.redbus.in/bus-tickets/delhi-to-har...,India Tours & Travels (GetBookCab),A/C Seater Push Back (2+2),22:00,06h 00m,04:00,1.7,INR 874 787,N/A
1181,Delhi to Haridwar Bus,https://www.redbus.in/bus-tickets/delhi-to-har...,Hans India Tour,A/C Seater / Sleeper (2+2),23:05,05h 40m,04:45,1.3,Starts from\nINR 1299,N/A


In [12]:
# read the csv files
path = r"Z:/Redbus project/UPSRTC(Uttar pradesh) bus details.csv"
df_upsrtc.to_csv(path,index = False)

# 9.Punjab Routes Bus details

In [3]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the PEPSU option
pepsu_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='PEPSU']")
driver.execute_script("arguments[0].click();", pepsu_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 4):
    if page_number < 4:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 30']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        PUNJAB.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
PUNJAB=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(3)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in PUNJAB:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/patiala-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-patiala', 'https://www.redbus.in/bus-tickets/ludhiana-to-delhi', 'https://www.redbus.in/bus-tickets/ludhiana-to-delhi-airport', 'https://www.redbus.in/bus-tickets/jalandhar-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-ludhiana', 'https://www.redbus.in/bus-tickets/jalandhar-to-delhi-airport', 'https://www.redbus.in/bus-tickets/delhi-to-jalandhar', 'https://www.redbus.in/bus-tickets/delhi-airport-to-patiala', 'https://www.redbus.in/bus-tickets/delhi-airport-to-ludhiana', 'https://www.redbus.in/bus-tickets/chandigarh-to-patiala', 'https://www.redbus.in/bus-tickets/delhi-to-amritsar', 'https://www.redbus.in/bus-tickets/phagwara-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-airport-to-jalandhar', 'https://www.redbus.in/bus-tickets/chandigarh-to-bathinda', 'https://www.redbus.in/bus-tickets/amritsar-to-delhi-airport', 'https://www.redbus.in/bus-tickets/patiala-to-amritsar', 'htt

In [4]:
df_punjab = pd.DataFrame(PUNJAB)
df_punjab

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,06h 09m,22:09,4.4,INR 711,N/A
1,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,06h 40m,22:40,3.8,INR 826,N/A
2,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66687,Volvo AC Seater (2+2),18:10,06h 09m,00:19,3.7,INR 711,N/A
3,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,Indo Canadian Transport Company,Mercedes A/C Semi Sleeper (2+2),18:00,07h 00m,01:00,4.2,INR 3300,N/A
4,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,Indo Canadian Transport Company,Volvo A/C Sleeper (2+1),18:00,08h 00m,02:00,4.2,INR 3850,N/A
...,...,...,...,...,...,...,...,...,...,...
682,Delhi to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-to-pha...,Kingfisher Busline Pvt Ltd,A/C Volvo B11R Multi-Axle Sleeper (2+1),21:00,07h 45m,04:45,3.0,Starts from\nINR 1350,N/A
683,Delhi to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-to-pha...,Big Bus,A/C Seater / Sleeper (2+2),17:35,06h 05m,23:40,2.9,Starts from\nINR 799 719,N/A
684,Delhi to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-to-pha...,CAPITAL HOLIDAYS,A/C Seater / Sleeper (2+2),17:00,07h 15m,00:15,2.8,Starts from\nINR 550 495,N/A
685,Delhi to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-to-pha...,Classic Travels,A/C Seater / Sleeper (2+2),21:35,07h 30m,05:05,2.6,INR 649,N/A


In [5]:
# read the csv files
path = r"Z:/Redbus project/PEPSU(Punjab) bus details.csv"
df_punjab.to_csv(path,index = False)

# 10. Bihar Routes Bus details

In [7]:
# Initialize the webdriver
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 30)

# Open the redBus website
driver.get('https://www.redbus.in/')
time.sleep(1)

#Scroll in one time
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(1)

 # Wait for the Arrow button to be clickable and then click it
driver.find_element(By.XPATH,"//span[@class='sc-hmzhuo dMsnin']").click()
time.sleep(1)

#Click on the PEPSU option
bsrtc_element = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='BSRTC']")
driver.execute_script("arguments[0].click();", bsrtc_element)


#Scroll function
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Wait for some time 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scroll to load all routes
scroll_page()
time.sleep(1)

# Collect route links
links = []
for page_number in range(1, 5):
    if page_number < 5:
        pagination_container = wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[12]"))
        )
        next_page = pagination_container.find_element(
            By.XPATH, (f".//div[contains(@class,'DC_117_pageTabs') and text()='{page_number}']")
        )
        actions.move_to_element(next_page).perform()
        time.sleep(1)
        next_page.click()
        time.sleep(1)
        scroll_page()
    
    link_elements = driver.find_elements(By.XPATH, ".//a[@class='route']")
    links.extend([element.get_attribute('href') for element in link_elements])

#Print the all route links
print(links)

# Collect the bus details function
def collection():
    time.sleep(1)
    for bus_detail in bus_details:
        try:
            try:
                bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
            except NoSuchElementException:
                bus_route_name = "N/A"

            try:
                bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
            except NoSuchElementException:
                bus_name = "N/A"

            try:
                bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
            except NoSuchElementException:
                bus_type = "N/A"

            try:
                departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
            except NoSuchElementException:
                departing_time = "N/A"

            try:
                duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
            except NoSuchElementException:
                duration = "N/A"

            try:
                reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
            except NoSuchElementException:
                reaching_time = "N/A"

            try:
                star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
            except NoSuchElementException:
                star_rating = "N/A"

            try:
                price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
            except NoSuchElementException:
                price = "N/A"

            try:
                seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top- 30']").text
            except NoSuchElementException:
                seat_availability = "N/A"
        except NoSuchElementException and TimeoutError:
            continue
        # Append the bus details in list
        BSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })
        
# Create a empty list
BSRTC=[]
# Visit each link and extract bus details
for link in links:
    driver.get(link)
    time.sleep(1)
    # Click the View buses  
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
        # Ensure the element is fully visible and not partially covered
               driver.execute_script("arguments[0].scrollIntoView(true);", button)
               WebDriverWait(driver, 10).until(EC.visibility_of(button))
               button.click()
               time.sleep(1)
            except (ElementClickInterceptedException, NoSuchElementException) as e:
                print(f"Click intercepted or element not found: {str(e)}")
    except:
        continue
    
    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True
    time.sleep(1)
    try:
        bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    except NoSuchElementException and TimeoutException:
        continue
    #Extract the bus details in function call
    collection()

#Print the list items
for bus in BSRTC:
    print(bus)

# Close the driver
driver.quit()

['https://www.redbus.in/bus-tickets/patna-to-motihari', 'https://www.redbus.in/bus-tickets/patna-to-bettiah', 'https://www.redbus.in/bus-tickets/bettiah-to-patna', 'https://www.redbus.in/bus-tickets/gopalganj-to-delhi', 'https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna', 'https://www.redbus.in/bus-tickets/patna-to-katihar', 'https://www.redbus.in/bus-tickets/motihari-to-delhi', 'https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar', 'https://www.redbus.in/bus-tickets/delhi-to-motihari', 'https://www.redbus.in/bus-tickets/muzaffarpur-to-ranchi', 'https://www.redbus.in/bus-tickets/patna-to-ranchi', 'https://www.redbus.in/bus-tickets/patna-to-kathmandu', 'https://www.redbus.in/bus-tickets/ranchi-to-patna', 'https://www.redbus.in/bus-tickets/hazaribagh-to-patna', 'https://www.redbus.in/bus-tickets/patna-to-purnea', 'https://www.redbus.in/bus-tickets/ranchi-to-muzaffarpur', 'https://www.redbus.in/bus-tickets/patna-to-hazaribagh', 'https://www.redbus.in/bus-tickets/ka

In [8]:
df_bsrtc = pd.DataFrame(BSRTC)
df_bsrtc

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Patna to Motihari Bus,https://www.redbus.in/bus-tickets/patna-to-mot...,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),21:20,03h 40m,01:00,4.0,INR 230,N/A
1,Patna to Motihari Bus,https://www.redbus.in/bus-tickets/patna-to-mot...,Siddhi Vinayak (BSRTC U/T),Non AC Seater (2+2),15:30,04h 00m,19:30,2.2,INR 220,N/A
2,Patna to Motihari Bus,https://www.redbus.in/bus-tickets/patna-to-mot...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:00,06h 00m,04:00,4.5,Starts from\nINR 550 495,N/A
3,Patna to Motihari Bus,https://www.redbus.in/bus-tickets/patna-to-mot...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,04h 55m,03:15,3.1,Starts from\nINR 550 495,N/A
4,Patna to Motihari Bus,https://www.redbus.in/bus-tickets/patna-to-mot...,Panwar Travels,A/C Seater / Sleeper (2+1),18:10,03h 25m,21:35,2.7,Starts from\nINR 750 713,N/A
...,...,...,...,...,...,...,...,...,...,...
393,Ranchi to Hazaribagh Bus,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Kaimur King,A/C Seater / Sleeper (2+1),20:15,03h 15m,23:30,1.8,Starts from\nINR 600,N/A
394,Ranchi to Hazaribagh Bus,https://www.redbus.in/bus-tickets/ranchi-to-ha...,BD Parivahan,A/C Seater / Sleeper (2+2),21:00,02h 50m,23:50,1.5,Starts from\nINR 1000,N/A
395,Ranchi to Hazaribagh Bus,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Sri Krishna Rath,A/C Seater / Sleeper (2+2),22:00,01h 15m,23:15,1.6,Starts from\nINR 1050,N/A
396,Ranchi to Hazaribagh Bus,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Maharani Travels,NON A/C Seater / Sleeper (2+2),22:30,01h 35m,00:05,2.9,Starts from\nINR 600,N/A


In [9]:
# read the csv files
path = r"Z:/Redbus project/BSRTC(Bihar) bus details.csv"
df_bsrtc.to_csv(path,index = False)